In [1]:
import pandas as pd

In [13]:
# Load datasets
traffic_df = pd.read_csv("Dataset_Uber Traffic.csv")
weather_df = pd.read_csv("Delhi_NCR_2015_2016.csv")
holiday_df = pd.read_csv("Holidays 2015 to 2016.csv")

In [14]:
# Convert traffic DateTime
traffic_df['DateTime'] = pd.to_datetime(traffic_df['DateTime'], format='%d/%m/%y %H:%M')

In [15]:
# Convert weather date
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%d-%m-%Y')

In [24]:
# Convert 'Date' column to datetime if it exists
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])
holiday_df['Only_Date'] = holiday_df['Date'].dt.date  # just the date part

In [25]:
# Preview the result
print("Traffic Date Range:", traffic_df['DateTime'].min(), "to", traffic_df['DateTime'].max())
print("Weather Date Range:", weather_df['date'].min(), "to", weather_df['date'].max())
print("Holiday Date Range:", holiday_df['Date'].min(), "to", holiday_df['Date'].max())

Traffic Date Range: 2015-11-01 00:00:00 to 2017-06-30 23:00:00
Weather Date Range: 2015-01-01 00:00:00 to 2016-12-31 00:00:00
Holiday Date Range: 2015-01-04 00:00:00 to 2016-12-13 00:00:00


In [26]:
# Step 1: Extract only date from traffic timestamp to match with daily weather data
traffic_df['Date'] = traffic_df['DateTime'].dt.date

In [27]:

# Step 2: Make sure weather date is also in same date format (date only)
weather_df['Date'] = weather_df['date'].dt.date

In [28]:
# Step 3: Merge traffic + weather on 'Date'
merged_df = pd.merge(traffic_df, weather_df, on='Date', how='left')

In [29]:
# Step 4: Drop the extra columns not needed
merged_df.drop(['date'], axis=1, inplace=True)

In [30]:

# Preview merged data
merged_df.head()

,DateTime,Junction,Vehicles,ID,Date,tavg,tmin,tmax,prcp
0,2015-11-01 00:00:00,1,15,20151101001,2015-11-01,22.9,16.2,31.4,NaN
1,2015-11-01 01:00:00,1,13,20151101011,2015-11-01,22.9,16.2,31.4,NaN
2,2015-11-01 02:00:00,1,10,20151101021,2015-11-01,22.9,16.2,31.4,NaN
3,2015-11-01 03:00:00,1,7,20151101031,2015-11-01,22.9,16.2,31.4,NaN
4,2015-11-01 04:00:00,1,9,20151101041,2015-11-01,22.9,16.2,31.4,NaN


In [32]:
holiday_df.rename(columns={'Holiday Date': 'Date'}, inplace=True)
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])
holiday_df['Only_Date'] = holiday_df['Date'].dt.date

In [33]:
# Step 2: Create a 'IsHoliday' flag
holiday_df['IsHoliday'] = 1

In [37]:
# Ensure both 'Date' columns are in datetime64[ns] format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])

# Now safely merge
merged_df = pd.merge(merged_df, holiday_df[['Date', 'IsHoliday']], on='Date', how='left')

In [38]:
# Step 4: Fill non-holiday entries with 0
merged_df['IsHoliday'] = merged_df['IsHoliday'].fillna(0).astype(int)

In [39]:
# Step 5: Preview the updated dataset
merged_df.head()

,DateTime,Junction,Vehicles,ID,Date,tavg,tmin,tmax,prcp,IsHoliday
0,2015-11-01 00:00:00,1,15,20151101001,2015-11-01,22.9,16.2,31.4,NaN,0
1,2015-11-01 01:00:00,1,13,20151101011,2015-11-01,22.9,16.2,31.4,NaN,0
2,2015-11-01 02:00:00,1,10,20151101021,2015-11-01,22.9,16.2,31.4,NaN,0
3,2015-11-01 03:00:00,1,7,20151101031,2015-11-01,22.9,16.2,31.4,NaN,0
4,2015-11-01 04:00:00,1,9,20151101041,2015-11-01,22.9,16.2,31.4,NaN,0


In [40]:
# ✅ Check for missing values in merged dataset
print("Missing Values:\n", merged_df.isnull().sum())

Missing Values:
 DateTime         0
Junction         0
Vehicles         0
ID               0
Date             0
tavg         17376
tmin         17808
tmax         17520
prcp         40920
IsHoliday        0
dtype: int64


In [46]:
#  Remove rows with missing 'Vehicles' (this is essential data)
merged_df = merged_df.dropna(subset=['Vehicles'])

In [47]:
# For weather columns (tavg, tmin, tmax, prcp), fill with median values
weather_cols = ['tavg', 'tmin', 'tmax', 'prcp']
for col in weather_cols:
    merged_df[col] = merged_df[col].fillna(merged_df[col].median())

In [48]:
# Remove duplicates
merged_df = merged_df.drop_duplicates()

In [49]:

# Ensure correct data types
merged_df['Vehicles'] = merged_df['Vehicles'].astype(int)
merged_df['IsHoliday'] = merged_df['IsHoliday'].astype(int)

In [50]:
#  Final check
print("\n Cleaned Data Types:\n", merged_df.dtypes)
print("\n Final Dataset Shape:", merged_df.shape)


 Cleaned Data Types:
 DateTime     datetime64[ns]
Junction              int64
Vehicles              int32
ID                    int64
Date         datetime64[ns]
tavg                float64
tmin                float64
tmax                float64
prcp                float64
IsHoliday             int32
dtype: object

 Final Dataset Shape: (48120, 10)


In [51]:
from sklearn.preprocessing import StandardScaler

# Step 1: Columns to scale
scale_cols = ['Vehicles', 'tavg', 'tmin', 'tmax', 'prcp']

In [52]:
# Step 2: Initialize the scaler
scaler = StandardScaler()

In [53]:
# Step 3: Apply scaler
merged_df[scale_cols] = scaler.fit_transform(merged_df[scale_cols])

In [54]:
# Step 4: Show first few rows
merged_df[scale_cols + ['Junction', 'IsHoliday', 'DateTime']].head()

,Vehicles,tavg,tmin,tmax,prcp,Junction,IsHoliday,DateTime
0,-0.375489,-0.559729,-0.420101,-0.274024,-0.161477,1,0,2015-11-01 00:00:00
1,-0.471875,-0.559729,-0.420101,-0.274024,-0.161477,1,0,2015-11-01 01:00:00
2,-0.616454,-0.559729,-0.420101,-0.274024,-0.161477,1,0,2015-11-01 02:00:00
3,-0.761034,-0.559729,-0.420101,-0.274024,-0.161477,1,0,2015-11-01 03:00:00
4,-0.664648,-0.559729,-0.420101,-0.274024,-0.161477,1,0,2015-11-01 04:00:00
